<a href="https://colab.research.google.com/github/romitbarua/MultiModalDeepFake/blob/main/TestingVideos/video_sampling_for_general_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import pandas as pd

In [3]:
metadata_file = r'/content/drive/MyDrive/DeepFakeDetection/data/FakeAVCeleb_v1.2/wav2lip_metadata_updated.csv'
metadata_df = pd.read_csv(metadata_file)

In [4]:
metadata_df.columns

Index(['video_id', 'source', 'target1', 'target2', 'method', 'category',
       'type', 'race', 'gender', 'filename', 'directory_path', 'mp4_filepath',
       'utterance_file', 'wav_filepath', 'audio_length', 'textgrid_filepath'],
      dtype='object')

In [11]:
mp4_real_proc = metadata_df[metadata_df.category=='A'].mp4_filepath

In [28]:
mp4_fake_proc = metadata_df[metadata_df.category!='A'].mp4_filepath.to_list()

## Dlib Video Testing

In [5]:
import os
import cv2
import dlib
from google.colab.patches import cv2_imshow
import numpy as np 
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import librosa
from scipy.io import wavfile
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor

In [6]:
SYM_PATH = '/content/drive/MyDrive/DeepFakeDetection'
%cd $SYM_PATH
%pip install -e .

/content/drive/MyDrive/DeepFakeDetection
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/MyDrive/DeepFakeDetection
  Running setup.py develop for DeepFake


In [7]:
from packages.DlibManager import DlibManager

predictor_path = '/content/drive/MyDrive/DeepFakeDetection/model/shape_predictor_68_face_landmarks.dat'
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

In [12]:
len(mp4_real_proc)

500

In [14]:
failed_real_mp4 = []

for idx, train_video_path in enumerate(mp4_real_proc):

  print(f'Video #{idx+1} out of {len(mp4_real_proc)}') 

  video = cv2.VideoCapture(train_video_path)

  try:
    dlib_video = DlibManager(predictor, detector, video)
  
  except:
    print(f'Failed to Upload: Video #{idx+1}')
    print(f'Failed for video path #{train_video_path}')
    failed_real_mp4.append(train_video_path)

  video.release()

Video #1 out of 500
Video #2 out of 500
Video #3 out of 500
Video #4 out of 500
Failed to Upload: Video #4
Failed for video path #/content/drive/MyDrive/DeepFakeDetection/data/FakeAVCeleb_v1.2/RealVideo-RealAudio/African/men/id00366/00118.mp4
Video #5 out of 500
Video #6 out of 500
Video #7 out of 500
Video #8 out of 500
Video #9 out of 500
Video #10 out of 500
Video #11 out of 500
Failed to Upload: Video #11
Failed for video path #/content/drive/MyDrive/DeepFakeDetection/data/FakeAVCeleb_v1.2/RealVideo-RealAudio/African/men/id00761/00072.mp4
Video #12 out of 500
Video #13 out of 500
Video #14 out of 500
Video #15 out of 500
Video #16 out of 500
Failed to Upload: Video #16
Failed for video path #/content/drive/MyDrive/DeepFakeDetection/data/FakeAVCeleb_v1.2/RealVideo-RealAudio/African/men/id01036/00010.mp4
Video #17 out of 500
Video #18 out of 500
Failed to Upload: Video #18
Failed for video path #/content/drive/MyDrive/DeepFakeDetection/data/FakeAVCeleb_v1.2/RealVideo-RealAudio/Africa

In [15]:
real_mp4_fav = [path for path in mp4_real_proc if path not in failed_real_mp4]

In [17]:
len(real_mp4_fav)

427

In [19]:
import csv

real_mp4_fav_path = '/content/drive/MyDrive/DeepFakeDetection/data/FakeAVCeleb_v1.2/training_metadata/real_mp4_fav_paths.csv'

with open(real_mp4_fav_path, 'w', newline='') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
     wr.writerow(real_mp4_fav)

In [36]:
working_fake_mp4 = {}

for idx, train_video_path in enumerate(mp4_fake_proc):

  print(f'Video #{idx+1} out of {len(mp4_fake_proc)}') 

  id = train_video_path.split('/')[-2]

  if id not in working_fake_mp4:

    video = cv2.VideoCapture(train_video_path)

    try:
      dlib_video = DlibManager(predictor, detector, video)

      working_fake_mp4[id] = train_video_path
    
    except:
      print(f'Failed to Upload: Video #{idx+1}')
      print(f'Failed for video path #{train_video_path}')
      continue

    video.release()
  
  else:
    continue

Streaming output truncated to the last 5000 lines.
Failed to Upload: Video #5348
Failed for video path #/content/drive/MyDrive/DeepFakeDetection/data/FakeAVCeleb_v1.2/FakeVideo-FakeAudio/African/men/id01717/00005_id01171_wavtolip.mp4
Video #5349 out of 9602
Failed to Upload: Video #5349
Failed for video path #/content/drive/MyDrive/DeepFakeDetection/data/FakeAVCeleb_v1.2/FakeVideo-FakeAudio/African/men/id01717/00005_id01392_wavtolip.mp4
Video #5350 out of 9602
Failed to Upload: Video #5350
Failed for video path #/content/drive/MyDrive/DeepFakeDetection/data/FakeAVCeleb_v1.2/FakeVideo-FakeAudio/African/men/id01717/00005_id01544_wavtolip.mp4
Video #5351 out of 9602
Failed to Upload: Video #5351
Failed for video path #/content/drive/MyDrive/DeepFakeDetection/data/FakeAVCeleb_v1.2/FakeVideo-FakeAudio/African/men/id01717/00005_id04727_wavtolip.mp4
Video #5352 out of 9602
Video #5353 out of 9602
Video #5354 out of 9602
Video #5355 out of 9602
Video #5356 out of 9602
Video #5357 out of 9602
V

In [38]:
len(working_fake_mp4)

457

In [40]:
len(set(working_fake_mp4.keys()))

457

In [42]:
fake_mp4_fav = list(working_fake_mp4.values())

In [43]:
import csv

fake_mp4_fav_path = '/content/drive/MyDrive/DeepFakeDetection/data/FakeAVCeleb_v1.2/training_metadata/fake_mp4_fav_paths.csv'

with open(fake_mp4_fav_path, 'w', newline='') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
     wr.writerow(fake_mp4_fav)